<a href="https://colab.research.google.com/github/yuliia-shevchenko137/yuliia-shevchenko137/blob/main/keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install corextopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import scipy.sparse as ss
import matplotlib.pyplot as plt

import corextopic.corextopic as ct
import corextopic.vis_topic as vt # jupyter notebooks will complain matplotlib is being loaded twice

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline

In [ ]:
import pandas as pd

In [ ]:
bel_list = list(pd.read_csv('belarus.csv', usecols=[0], names=['bel']).bel)

In [ ]:
bel_list

In [ ]:
# Transform 20 newsgroup data into a sparse matrix
vectorizer = CountVectorizer(max_features=20000, binary=True)
doc_word = vectorizer.fit_transform(bel_list)
doc_word = ss.csr_matrix(doc_word)

doc_word.shape # n_docs x m_words

(988, 4209)

In [ ]:
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names()))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
words

In [ ]:
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]

doc_word.shape # n_docs x m_words

(988, 4156)

In [ ]:
# Train the CorEx topic model with 50 topics
topic_model = ct.Corex(n_hidden=10, words=words, max_iter=200, verbose=False, seed=1)
topic_model.fit(doc_word, words=words);

In [ ]:
topic_model.get_topics(topic=1, n_words=10)

[('другому', 0.07297897841235255, 1.0),
 ('оператору', 0.07231687367515822, 1.0),
 ('отсутствие', 0.02205116708761179, 1.0),
 ('вся', 0.020489493033539214, 1.0),
 ('безлимитище', 0.020489493033539214, 1.0),
 ('семья', 0.020489493033539214, 1.0),
 ('перевести', 0.0174596432317513, 1.0),
 ('кабинет', 0.014440107196223037, 1.0),
 ('через', 0.014246087091660252, 1.0),
 ('тоже', 0.013037887554187567, 1.0)]

In [ ]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ', '.join(topic_words))

0: когда, по, churn_single_model, раньше, not_churn, был, эти, там, сим, смс
1: другому, оператору, отсутствие, вся, безлимитище, семья, перевести, кабинет, через, тоже
2: планы, тарифные, уровне, план, тарифный, ответы, вопросы, центре, колл, дают
3: на, мтс, не, интернет, переходить, если, то, перейти, бы, номер
4: ещё, предложили, тарифного, надеюсь, никаких, плана, тех, условиях, проблема, вообще
5: так, как, же, будет, интернета, чтобы, минус, при, теперь, до
6: другого, оператора, но, нет, из, мне, вас, ни, меня, для
7: за, лет, уже, это, компании, вашей, услугами, много, услуги, другую
8: что, деньги, или, вам, от, этого, оператор, безлимит, того, комфорт
9: время, долгое, ожидание, ваши, ожидания, ближайшее, соединения, длительное, своим, выбор


In [ ]:
tfidf

<988x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 11074 stored elements in Compressed Sparse Row format>

In [ ]:
from nltk import pos_tag, word_tokenize
word_tokenize("John's big idea isn't all that bad.")

['John', "'s", 'big', 'idea', 'is', "n't", 'all', 'that', 'bad', '.']

In [ ]:
pos_tag(['car'])[0][1]

'NN'

In [ ]:
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger_ru")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.


True

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    #tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in word_tokenize(text) if token not in russian_stopwords\
              and token != " " \
              and pos_tag([token],  lang='rus')[0][1] == 'S' \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
bel_list = [preprocess_text(i) for i in bel_list]

In [ ]:
bel_list[0]

'лет компанией Сотрудники надежда предложение подробности предложения А1 компания рынке связи услугами связи интернета телевидения'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, )
tfidf = tfidf_vectorizer.fit_transform(bel_list)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(bel_list)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 7

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
print("______________")
display_topics(lda, tf_feature_names, no_top_words)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,


Topic 0:
мтс тариф условия номер месяц плата друзья интернета время лет
Topic 1:
связь а1 лет тарифы операторы лайф велком возможности консультанты городом
Topic 2:
обслуживание уровне клиентов консультация выбор цены удачи условия номера план
Topic 3:
качество связи обслуживания услуг сотрудники интернета покрытие скорость а1 оператором
Topic 4:
интернет комфорт лайф звонки деньги трафик сеть сети оператору компанию
Topic 5:
тарифы оператора ответа ответ время минут ожидания ожидание интернета переход
Topic 6:
оператор вопросы сотрудники вопрос ответы информацию деньги велком услуга рекомендации
______________
Topic 0:
интернет связь тарифы мтс а1 оператору оператора оператор сети лайф
Topic 1:
оператора ответа минут время ожидание ожидания оператором номер соединения мин
Topic 2:
обслуживание вопросы уровне ответы план выбор консультация клиентов работу вежливость
Topic 3:
процветания удачи работники атмосфера услуги обслуживании позиции высоте пользователь ассортимент
Topic 4:
ответ

# YAKE!

In [ ]:
pip install yake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 4.5 MB/s 
     |████████████████████████████████| 132 kB 20.4 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73984 sha256=8ba5a39c236d4f06923f8c271dab5f6d8bfb838f9ee6c3dccf6f1c082197593e
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [ ]:

import yake
kw_extractor = yake.KeywordExtractor()
text = " ".join(bel_list)
language = "ru"
max_ngram_size = 1
deduplication_threshold = 0.4
numOfKeywords = 30
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
  print(kw)

('МТС', 0.0004231757946718164)
('интернет', 0.0012179799999456366)
('велком', 0.0016079027229189708)
('связь', 0.0017072578163059501)
('оператора', 0.0020424256190803534)
('качество', 0.0028920514759531155)
('тарифы', 0.0029269965255613226)
('обслуживание', 0.003210693014711789)
('быстро', 0.003964456488929722)
('переходить', 0.005654527676357303)
('пользуюсь', 0.006768781980998257)
('хорошая', 0.007179400431087089)
('минут', 0.007778696556885282)
('услуг', 0.008595372609628503)
('Вашей', 0.00916773493745987)
('другому', 0.009426368671524537)
('лайф', 0.009998634278468038)
('вопросы', 0.010835299658029575)
('ответ', 0.011033636310816188)
('сети', 0.012040047931423747)
('устраивает', 0.012087273235095595)
('деньги', 0.012955846745847465)
('вежливый', 0.013204461006900804)
('нравится', 0.013324535141791934)
('цены', 0.014334765263048974)
('месяц', 0.014595089662720575)
('номер', 0.015096786442482548)
('удобно', 0.01542775716419964)
('понятно', 0.017111585656948737)
('Думаю', 0.0177252153

In [ ]:
import os
dir_path = os.path.dirname(os.path.realpath('__file__'))

local_path = os.path.join("StopwordsList", "stopwords_%s.txt" % 'ru'[:2].lower())


resource_path = os.path.join(dir_path,local_path)

with open(resource_path, encoding='utf-8') as stop_fil:
  stopword_set = set( stop_fil.read().lower().split("\n") )

In [ ]:
stopword_set

In [ ]:

text = " ".join(bel_list).lower()
language = "ru"
max_ngram_size = 2
deduplication_algo = 'jaro'
deduplication_threshold = 0.8
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                            dedupLim=deduplication_threshold, dedupFunc='jaro',
                                            top=numOfKeywords, features=["WRel", "WFreq", "WPos","KPF"],
                                            windowsSize=5, stopwords=stopword_set)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
  print(kw)

('мтс', 0.0008997600827391973)
('качество связи', 0.0012609653665633213)
('интернет', 0.0013194107307956074)
('связь', 0.0017622073516084305)
('оператора', 0.0021535801571730033)
('другого оператора', 0.002758077074091784)
('тарифы', 0.0030385069095667182)
('велком', 0.0030976213224561326)
('обслуживание', 0.003449968116778394)
('хорошая связь', 0.003925081631467195)
('безлимитный интернет', 0.004896781925725143)
('быстро', 0.0055331981887819836)
('переходить', 0.005917454710525366)
('ответа оператора', 0.006413504365985597)
('минут', 0.00756996948668519)
('пользуюсь', 0.007836055481782367)
('хорошее обслуживание', 0.007913104195268476)
('услуг', 0.008588369859214361)
('другому', 0.010150690237325783)
('вопрос', 0.011448559125042198)


In [ ]:

text = " ".join(bel_list).lower()
language = "ru"
max_ngram_size = 2
deduplication_algo = 'jaro'
deduplication_threshold = 0.4
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                            dedupLim=deduplication_threshold, 
                                            top=numOfKeywords, #features=["WRel", "WFreq", "WPos","KPF"],
                                            windowsSize=5)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
  print(kw)

('мтс', 0.0008949212295442503)
('качество связи', 0.0012486526315953262)
('интернет', 0.0013125567333926913)
('связь', 0.001753682612805139)
('оператора', 0.0021418880723940758)
('тарифы', 0.003023181100309279)
('велком', 0.0030820850780121285)
('обслуживание', 0.0034359305652211253)
('хорошая связь', 0.0038927680903168603)
('безлимитный интернет', 0.004849841367837688)
('быстро', 0.005512764793800913)
('переходить', 0.005888699134460813)
('минут', 0.0075329968220186816)
('пользуюсь', 0.007802044897155099)
('услуг', 0.008549976623863451)
('другому', 0.010103599921593966)
('ответ', 0.01139350818662601)
('вопрос', 0.011400360719090194)
('сети', 0.011989971586537055)
('лайф', 0.012756642221746906)


In [ ]:

text = " ".join(bel_list).lower()
language = "ru"
max_ngram_size = 2
deduplication_algo = 'jaro'
deduplication_threshold = 0.4
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                            dedupLim=deduplication_threshold, 
                                            top=numOfKeywords, #features=["WRel", "WFreq", "WPos","KPF"],
                                            windowsSize=5)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
  print(kw)

('мтс', 0.0008997600827391973)
('качество связи', 0.0012609653665633213)
('интернет', 0.0013194107307956074)
('связь', 0.0017622073516084305)
('оператора', 0.0021535801571730033)
('тарифы', 0.0030385069095667182)
('велком', 0.0030976213224561326)
('обслуживание', 0.003449968116778394)
('хорошая связь', 0.003925081631467195)
('безлимитный интернет', 0.004896781925725143)
('быстро', 0.0055331981887819836)
('переходить', 0.005917454710525366)
('минут', 0.00756996948668519)
('пользуюсь', 0.007836055481782367)
('услуг', 0.008588369859214361)
('другому', 0.010150690237325783)
('ответ', 0.011444898415919781)
('вопрос', 0.011448559125042198)
('сети', 0.012040004025872202)
('лайф', 0.012808817330781846)


In [ ]:
it_list = list(pd.read_csv('/content/it_no_punc.csv', usecols=[0], names=['it']).it)
text = " ".join(it_list)
language = "it"
max_ngram_size = 2
deduplication_algo = 'jaro'
deduplication_threshold = 0.4
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                            dedupLim=deduplication_threshold, 
                                            top=numOfKeywords, #features=["WRel", "WFreq", "WPos","KPF"],
                                            windowsSize=5)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
  print(kw)

('sicurezza', 2.517761563463938e-05)
('ottimo servizio', 7.861474028884633e-05)
('servizio clienti', 8.445511356091763e-05)
('nessun problema', 8.997724700066074e-05)
('carta', 9.482373899769532e-05)
('facile', 0.00010531316396219786)
('Nexi', 0.00017100134427836712)
('facile utilizzo', 0.00022239415269670535)
('APP', 0.00025302581117373987)
('nessun', 0.0003253159269466443)
('sito', 0.00035779302323700896)
('ottimo', 0.0003809854939759937)
('spese', 0.00040048930017975423)
('pos', 0.0004032650448820267)
('assistenza', 0.0005580065018137171)
('banca', 0.0007009898799221393)
('costi', 0.0007250986182747074)
('call center', 0.0009026818213293739)
('riesco', 0.0009225419137158242)
('pagamenti', 0.0010092594342537124)


In [ ]:
list([kw[0] for kw in keywords])

['sicurezza',
 'ottimo servizio',
 'servizio clienti',
 'nessun problema',
 'carta',
 'facile',
 'Nexi',
 'facile utilizzo',
 'APP',
 'nessun',
 'sito',
 'ottimo',
 'spese',
 'pos',
 'assistenza',
 'banca',
 'costi',
 'call center',
 'riesco',
 'pagamenti']

In [ ]:
lhg_list = list(pd.read_csv('/content/LHG_unprocessed (1).csv', usecols=[0], names=['lhg']).lhg)

In [ ]:
text = " ".join(lhg_list)
language = "en"
max_ngram_size = 1
deduplication_algo = 'jaro'
deduplication_threshold = 0.9
numOfKeywords = 15
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                            dedupLim=deduplication_threshold, 
                                            
                                            top=numOfKeywords, features=None, windowsSize=10)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
  print(kw)

('airport', 0.00012460612986349046)
('lounge', 0.0001624386830716363)
('Lufthansa', 0.00032172712156714405)
('lounges', 0.0009404344809410524)
('Lufthansas', 0.0009973540768581468)
('check', 0.0011358792480629138)
('terminal', 0.0012242528235854198)
('boarding', 0.0013480365880011354)
('bags', 0.0014110554212595626)
('flight', 0.001412112270011924)
('airports', 0.0014225866492748496)
('check-in', 0.001688825131874932)
('time', 0.001852514893736633)
('security', 0.001856124726306618)
('bag', 0.001901211514960253)


In [ ]:
text = " ".join(lhg_list)
language = "en"
max_ngram_size = 2
deduplication_algo = 'jaro'
deduplication_threshold = 0.45
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                            dedupLim=deduplication_threshold, 
                                            top=numOfKeywords, features=["WRel", "WFreq","WSpread", "WPos","KPF"],
                                            windowsSize=1)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
  print(kw)

('airport', 0.000179796108538962)
('lounge', 0.00023222995864914434)
('check', 0.0012150204172653605)
('Lufthansa', 0.0013026714723715772)
('customer service', 0.0013298153407195912)
('bags', 0.001454970907186185)
('boarding', 0.0014969090406193055)
('flight', 0.0015010792388149897)
('time', 0.001865224200065368)
('security', 0.0019596873617177763)
('luggage', 0.002014929059142515)
('terminal', 0.002016537066167246)
('service', 0.0024748457555406863)
('arrived', 0.002525706253102061)
('online check-in', 0.0037737571207540205)
('gate', 0.0037952575814987843)
('online', 0.003898275881181252)
('staff', 0.00417854510403737)
('business lounge', 0.004251972226183939)
('food', 0.004904114496635619)


In [ ]:
text = " ".join(lhg_list)
language = "en"
max_ngram_size = 2
deduplication_algo = 'jaro'
deduplication_threshold = 0.45
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                            dedupLim=deduplication_threshold, 
                                            top=numOfKeywords, features=["WRel", "WFreq","WSpread", "WCase", "WPos","KPF"],
                                            windowsSize=1)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
  print(kw)

('airport', 0.00013566606066446897)
('lounge', 0.00016592188729439917)
('Lufthansa lounge', 0.0003109858043879286)
('check', 0.001104195722885554)
('Frankfurt airport', 0.0011679223564490635)
('customer service', 0.001194260500042179)
('terminal', 0.0012388821158954718)
('boarding', 0.001365851123177384)
('bags', 0.0014348114383042226)
('flight', 0.0014525074726090405)
('Business Class', 0.0017233667429624852)
('time', 0.001865224200065368)
('security', 0.001875869251776261)
('Air France', 0.001889968699285463)
('luggage', 0.0019708364020430288)
('arrived', 0.0022318373837835596)
('service', 0.002312015864729628)
('Air', 0.002575771353808585)
('Class', 0.003061978645606653)
('online check-in', 0.0032267580583843613)


In [ ]:
pip install rake_nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from rake_nltk import Rake
rake_nltk_var = Rake()

rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)


['unrefrigerated ‚ äö √ ñ √∫ iced ‚ äö √ ñ √ π tea', 'security check meaning leaving ‚ äö √ ñ √≤ airside ‚ äö √ ñ √¥', 'always ‚ äö √ ñ √≤ surprised ‚ äö √ ñ √¥', 'appalling poor customer service massive queues poor signs limited seating rude security staff long queues everywhere', 'air france ‚ äö √ ñ √¥', 'mcdonalds wendys subway taco maker 5 cafeterias ... also', 'earlier problems poor air conditioning gloomy corridor like interior dreadful toilets', 'disabled ‚ äö √ ñ √¨', 'make sense terrible rental car airport_general unhappy staff poor customer service attitude', 'cramped poor food selection beyond carrot sticks tomatoes soup', 'gem ‚ àö ¬∫ tlich customer service', 'coffee tea bottled drinks alcoholic drinks greek wine', 'could use like united airlines lounge air canada maple leaf lufthansa senators lounge', 'tickets carpeted floor free wifi internet expensive gift shops', 'food sandwiches yogurt cheese cheese pie fruits', 'asian tigers singapore hong kong seoul etc', 'london bo

In [ ]:
from gensim.summarization import keywords

print(keywords(text))

airport
airports
lounge
lounges
check
checks
checking
lufthansa
lufthansas
flight
flights
checked bag
bags
terminal
terminals
terminalitied
boarded
board
boards
arrive
arriving
arrival
arrived
arrivals
arrives
security
secure
seat
seats
seated
luggage
luggages
time
times
timely
timed
service
services
servicing
staff
customs
customer
customers
custom
airline
airlines
boarding pass
passenger
hours
good
goods
baggage
likely
like
people
peoples
seating rude
completed
complete
air
aires
online
free
booked
book
booking
gates
gate
areas
international area
queue
internal
better
minutes
minute
minut
minuted
longe
travellers
travelling
traveller
travel
traveling
traveler
travelled
traveled
travels
travelers
news
connection
connecting
connect
connections
connected
connects
connectivity
completely new experience
hour delay
small
departure
departures
final
finale
little finally
waiting
wait
waited
nice
nicely
food means
class
classes
experiences
included
including
includes
bus
poor
use
uses
useful
